**INTRODUCTION**

Just over a year ago you had a smart electricity meter installed at your home. This particular model of smart meter reports your usage to your electricity provider every hour. Your electricity contract is due for renewal and being a top financial analyst you decide to identify which contract will minimize your annual electricity cost.


There are 3 different types of electricity contracts available to you:

• No Flex: The cost per kWh of electricity is constant for the entire year

• Monthly Flex: The cost per kWh of electricity fluctuates depending on the month

• Hourly Flex: The cost per kWh of electricity fluctuates based on the time of day


To decide which contract is optimal, you decide to compare the costs under each of the contracts assuming your usage remains exactly the same as last year. You've contacted your current electricity supplier to obtain your electricity usage history for the last year. Unfortunately, they provided it to you in a poorly structured and unsorted format, so you will need to clean it up prior to doing your analysis. In the data the hour field identifies when the hour starts, so 8AM would be from 8:00AM until 8:59AM.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel("data_cleaning.xlsx", sheet_name = 'Usage', header=None)

df.rename({0: 'raw_data'}, axis=1, inplace=True)

df

,raw_data
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [ ]:
df['raw_data'] = df['raw_data'].str.lower().str.strip()

df.head()

,raw_data
0,3 pm mon 24th-mar-2014___0.384 kwh
1,5am 15-aug-2014___1.201 kwh
2,__8pm thu 20-mar-2014____1.523 kwh
3,6pm 23rd-apr-2014___0.424 kwh
4,_1am friday 19th-dec-2014___0.209 kwh


In [ ]:
# hours
pattern_hours = re.compile(r"(?:[\s\_]*)(\d+)")
hours = df['raw_data'].str.extract(pattern_hours)
hours = hours.rename({0: 'hours'}, axis=1)

# am | pm
pattern_am_pm = re.compile(r"(am|pm)")
am_pm = df['raw_data'].str.extract(pattern_am_pm)
am_pm = am_pm.rename({0: 'am_pm'}, axis=1)
am_pm

# dates
pattern_date = r'(\d{1,2}\w+-\w+-\d{4})'
date = df['raw_data'].str.extract(pattern_date)
date = date.rename({0: 'date'}, axis=1)

# null dates
pattern_date1 = r'(\d{1,2}-[A-Za-z]+-\d{4})'
date_1 = df['raw_data'].loc[date[date['date'].isnull()].index].str.extract(pattern_date1)
date_1.rename({0: 'date'}, axis=1, inplace=True)

# concating dates with dates which were null previously
date.fillna(date_1, inplace=True)

# kwh
pattern_kwh = re.compile(r'(\d+\.\d+\s*kwh)')
kwh = pd.DataFrame(df['raw_data'].str.replace(" ", "").str.extract(pattern_kwh)[0].str.replace('kwh', ''))
kwh.rename({0: 'kwh'}, axis=1, inplace=True)

data = pd.concat([hours, am_pm, date, kwh], axis=1)
data

,hours,am_pm,date,kwh
0,3,pm,24th-mar-2014,0.384
1,5,am,15-aug-2014,1.201
2,8,pm,20-mar-2014,1.523
3,6,pm,23rd-apr-2014,0.424
4,1,am,19th-dec-2014,0.209
...,...,...,...,...
8755,1,am,07th-nov-2014,0.084
8756,6,am,20-may-2014,1.027
8757,2,am,8th-apr-2014,0.052
8758,9,pm,27th-jan-2014,1.428


Extracting day, month and year from the 'date' column.

In [ ]:
# day
pattern_day = r"(\d+)"
day = data['date'].str.extract(pattern_day)
day = day.rename({0: 'day'}, axis=1)

# month
pattern_month = r"(?:(?:\d{1,2}(?:st|nd|rd|th)-)?([A-Za-z]+)-\d{4})"
month = data['date'].str.extract(pattern_month)
month.rename({0: 'month'}, axis=1, inplace=True)

# year
pattern_year = r"(\d{4})"
year = data['date'].str.extract(pattern_year)
year.rename({0: 'year'}, axis=1, inplace=True)

# merging all columns into one dataframe
df_date = pd.concat([day, month, year], axis=1)
result = pd.concat([data, df_date], axis=1).drop('date', axis=1)
result['kwh'] = result['kwh'].astype('float')

result

,hours,am_pm,kwh,day,month,year
0,3,pm,0.384,24,mar,2014
1,5,am,1.201,15,aug,2014
2,8,pm,1.523,20,mar,2014
3,6,pm,0.424,23,apr,2014
4,1,am,0.209,19,dec,2014
...,...,...,...,...,...,...
8755,1,am,0.084,07,nov,2014
8756,6,am,1.027,20,may,2014
8757,2,am,0.052,8,apr,2014
8758,9,pm,1.428,27,jan,2014


In [ ]:
# convert columns into date type
result['date'] = result['day'] + result['month'] + result['year']
result['date'] = (result['date'] + result['hours'] + result['am_pm']).\
apply(lambda x: datetime.strptime(x, "%d%b%Y%I%p"))

# weekday
result['weekday'] = result['date'].dt.strftime("%A")

# month names
result['month_name'] = result['date'].dt.month_name()

result

,hours,am_pm,kwh,day,month,year,date,weekday,month_name
0,3,pm,0.384,24,mar,2014,2014-03-24 15:00:00,Monday,March
1,5,am,1.201,15,aug,2014,2014-08-15 05:00:00,Friday,August
2,8,pm,1.523,20,mar,2014,2014-03-20 20:00:00,Thursday,March
3,6,pm,0.424,23,apr,2014,2014-04-23 18:00:00,Wednesday,April
4,1,am,0.209,19,dec,2014,2014-12-19 01:00:00,Friday,December
...,...,...,...,...,...,...,...,...,...
8755,1,am,0.084,07,nov,2014,2014-11-07 01:00:00,Friday,November
8756,6,am,1.027,20,may,2014,2014-05-20 06:00:00,Tuesday,May
8757,2,am,0.052,8,apr,2014,2014-04-08 02:00:00,Tuesday,April
8758,9,pm,1.428,27,jan,2014,2014-01-27 21:00:00,Monday,January


### 1. What is your average hourly electricity usage?

In [ ]:
print(round(result['kwh'].mean(), 4), 'kWh')

0.7817 kWh


### 2. What is your average electricity usage per hour in February?

In [ ]:
print(round(result[result['month_name'] == 'February']['kwh'].mean(), 4), 'kWh')

0.8337 kWh


### 3. Which day of the week has the highest average usage?

In [ ]:
grouped_df = result.groupby('weekday')

# for key, item in grouped_df:
#     print(grouped_df.get_group(key), "\n\n")

round(grouped_df.mean(numeric_only=True).sort_values('kwh').tail(1), 4)

,kwh
weekday,
Sunday,0.8986


### 4. What is the highest amount of electricity used in a continuous 4 hour period?

In [ ]:
print(round(result.sort_values('date')['kwh'].rolling(window=4).sum().sort_values(ascending=False).head(1).values[0], 3), 'kWh')

17.237 kWh


### 5. Based on your historic electricity usage, what would your annual cost of electricity be under the "Monthly Flex" contract?

In [ ]:
month_mapping = {
    'January': 0.20,
    'February': 0.19,
    'March': 0.17,
    'April': 0.18,
    'May': 0.22,
    'June':	0.27,
    'July': 0.24,
    'August': 0.19,
    'September': 0.18,
    'October': 0.15,
    'November': 0.14,
    'December': 0.19
}

print('Annual cost of electricity under Monthly Flex Contract:', round((result['month_name'].map(month_mapping) * result['kwh']).sum(), 2), '$')

Annual cost of electricity under Monthly Flex Contract: 1421.21 $


### 6. Based on your historic electricity usage, which of the three contracts would produce the lowest annual cost?

In [ ]:
noflex_mapping = {
    'January': 0.21,
    'February': 0.21,
    'March': 0.21,
    'April': 0.21,
    'May': 0.21,
    'June':	0.21,
    'July': 0.21,
    'August': 0.21,
    'September': 0.21,
    'October': 0.21,
    'November': 0.21,
    'December': 0.21
}

In [ ]:
month_mapping = {
    'January': 0.20,
    'February': 0.19,
    'March': 0.17,
    'April': 0.18,
    'May': 0.22,
    'June':	0.27,
    'July': 0.24,
    'August': 0.19,
    'September': 0.18,
    'October': 0.15,
    'November': 0.14,
    'December': 0.19
}

In [ ]:
hour_am_mapping = {
    1: 0.10,
    2: 0.10,
    3: 0.10,
    4: 0.15,
    5: 0.20,
    6:	0.24,
    7: 0.24,
    8: 0.26,
    9: 0.20,
    10: 0.20,
    11: 0.26,
    12: 0.10
}

hour_pm_mapping = {
    1: 0.20,
    2: 0.24,
    3: 0.18,
    4: 0.15,
    5: 0.30,
    6:	0.24,
    7: 0.24,
    8: 0.12,
    9: 0.11,
    10: 0.10,
    11: 0.10,
    12: 0.26
}

In [ ]:
result['hours'] = result['hours'].astype(int)

print("AM mapping:")
print(result[result['am_pm'] == 'am']['hours'].map(hour_am_mapping))

print("\nPM mapping:")
print(result[result['am_pm'] == 'pm']['hours'].map(hour_pm_mapping))

In [ ]:
cost_am = round((result[result['am_pm'] == 'am']['hours'].map(hour_am_mapping) * result[result['am_pm'] == 'am']['kwh']).sum(), 2)
cost_pm = round((result[result['am_pm'] == 'pm']['hours'].map(hour_pm_mapping) * result[result['am_pm'] == 'pm']['kwh']).sum(), 2)

#print("Annual cost for AM: $", cost_am)
#print("Annual cost for PM: $", cost_pm)

In [ ]:
print('Annual cost of electricity under No Flex Contract:', round((result['month_name'].map(noflex_mapping) * result['kwh']).sum(), 2), '$')
print('Annual cost of electricity under Monthly Flex Contract:', round((result['month_name'].map(month_mapping) * result['kwh']).sum(), 2), '$')
print('Annual cost of electricity under Hourly Flex Contract:', cost_am+cost_pm, '$')

Annual cost of electricity under No Flex Contract: 1438.1 $
Annual cost of electricity under Monthly Flex Contract: 1421.21 $
Annual cost of electricity under Hourly Flex Contract: 1368.98 $


Results show that the best choice is: **Hourly Flex Contract**